In [1]:
from music21 import *
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import torch.nn.functional as F
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
import pandas as pd
from ast import literal_eval

notes_df = pd.read_csv ('Dataset/notes.csv')
train_df = pd.read_csv ('Dataset/trainset.csv')
validation_df = pd.read_csv ('Dataset/validationset.csv')

data_train = train_df[['x_tr','future']].to_numpy()
data_validation = validation_df[['x_val','future']].to_numpy()

x_tr_string = data_train[:,0]
y_tr_string = data_train[:,1]
x_tr = []
y_tr = []
for i in x_tr_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_x_tr = [int(j) for j in i.split()]
    x_tr.append(input_x_tr)

for i in y_tr_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_y_tr = [int(j) for j in i.split()]
    y_tr.append(input_y_tr)
x_tr = np.array(x_tr)
y_tr = np.array(y_tr)
    
# print(x_tr.shape,y_tr.shape)

x_val_string = data_validation[:,0]
y_val_string = data_validation[:,1]
x_val = []
y_val = []
for i in x_val_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_x_val = [int(j) for j in i.split()]
    x_val.append(input_x_val)

for i in y_val_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_y_val = [int(j) for j in i.split()]
    y_val.append(input_y_val)
x_val = np.array(x_val)
y_val = np.array(y_val)

# print(x_val.shape,y_val.shape)

notes_ = notes_df.to_numpy()[:,1]
unique_notes_reverse = dict(enumerate(notes_.flatten(), 1))
unique_notes = {value : key for (key, value) in unique_notes_reverse.items()}

In [3]:
from MusicDataset import *
batch_size = 8
train_set = MusicDataset(x_tr,y_tr)
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [4]:
val_set = MusicDataset(x_val,y_val)
validationloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

**DataLoader**

**For Wavenet Model (Ignore for now)**

In [5]:
from Models import Wavenet, LSTM
Net = Wavenet.Wavenet(len(unique_notes),blocks=4,layers = 2)
# Net = LSTM.Sequence(len(unique_notes),32)
Net.to(device)

Wavenet(
  (relu): ReLU()
  (Sigmoid): Sigmoid()
  (Tanh): Tanh()
  (embedding): Embedding(182, 100)
  (init_conv1d): Conv1d(100, 32, kernel_size=(1,), stride=(1,))
  (end_conv1d): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
  (end_conv2d): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=182, bias=True)
)

# Checking the model

In [6]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

**Use the line below when you want to change learning parameters in between**

In [7]:
import os
import time

baseDir = r'C:\Users\Dell\music_generation/Music-Gen-AI'
expDir = os.path.join(baseDir, 'trained_model_cache', time.strftime("%d_%m_%Y_%H_%M"))
lossDir = os.path.join(expDir, 'loss')
os.makedirs(expDir, exist_ok=True)
os.makedirs(lossDir, exist_ok=True)
print(baseDir, expDir)

C:\Users\Dell\music_generation/Music-Gen-AI C:\Users\Dell\music_generation/Music-Gen-AI\trained_model_cache\21_04_2024_11_42


In [8]:
loss_history = []
best_val_acc = 0
training_mode_list = ["Next_Predict_Mode","Future_Predict_Mode"]
training_mode = "Next_Predict_Mode"
# training_mode = "Future_Predict_Mode"
future_preds = 5

In [10]:
for epoch in range(30): 
    running_loss = 0.0
    start = timeit.default_timer()
    for i, data in enumerate(trainloader, 0):
        input, label = data
        input = input.to(device)  # Move input to device here
        optimizer.zero_grad()
        if training_mode == training_mode_list[0]:
            output = Net(input)
            loss = criterion(output, label[:, 0].long().to(device))
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * input.shape[0]  # Multiply by batch size
            total_preds += input.shape[0]
            _, predicted = torch.max(output, 1)
            correct_preds += (predicted == label[:, 0].to(device)).sum().item()
        elif training_mode == training_mode_list[1]:
            cumm_output = torch.zeros(0, len(unique_notes)).to(device)
            cumm_label = []
            for k in range(future_preds):
                output = Net(input)
                cumm_output = torch.cat((cumm_output, output))
                cumm_label.extend(label[:, k].tolist())
                next_preds = torch.max(output, 1)[1]
                input = torch.cat((input[:, 1:], next_preds.unsqueeze(1)), dim=1)
            loss = criterion(cumm_output, torch.tensor(cumm_label, dtype=torch.long).to(device))
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * input.shape[0]  # Multiply by batch size

        if (i % 300 == 299):
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / total_preds))
            loss_history.append(running_loss / total_preds)
            running_loss = 0.0
    stop = timeit.default_timer()
    print(f"Training Time for Epoch :{epoch} \n {stop - start} seconds\n")
    train_acc = correct_preds / total_preds * 100
    trainreport = "Training Accuracy : \n correct predictions  : {} \n total predictions : {} \n Training Accuracy : {} \n ------------------------\n".format(
        correct_preds, total_preds, train_acc)
    print(trainreport)
    correct_preds = 0
    total_preds = 0

    val_loss = 0
    start = timeit.default_timer()
    for i, data in enumerate(validationloader, 0):
        input, label = data
        input = input.to(device)  # Move input to device here
        if training_mode == training_mode_list[0]:
            output = Net(input)
            loss = criterion(output, label[:, 0].long().to(device))
            val_loss += loss.item() * input.shape[0]  # Multiply by batch size
            total_preds += input.shape[0]
            _, predicted = torch.max(output, 1)
            correct_preds += (predicted == label[:, 0].to(device)).sum().item()
        elif training_mode == training_mode_list[1]:
            cumm_output = torch.zeros(0, len(unique_notes)).to(device)
            cumm_label = []
            for k in range(future_preds):
                output = Net(input)
                cumm_output = torch.cat((cumm_output, output))
                cumm_label.extend(label[:, k].tolist())
                next_preds = torch.max(output, 1)[1]
                input = torch.cat((input[:, 1:], next_preds.unsqueeze(1)), dim=1)
            loss = criterion(cumm_output, torch.tensor(cumm_label, dtype=torch.long).to(device))
            val_loss += loss.item() * input.shape[0]  # Multiply by batch size

    stop = timeit.default_timer()
    print(f"Validation Time for Epoch :{epoch} \n {stop - start} seconds\n")
    val_acc = correct_preds / total_preds * 100
    val_loss = val_loss / total_preds
    valreport = "Validation Accuracy : \n correct predictions  : {} \n total predictions : {} \n Val-Loss : {} \n Val-Accuracy :{}----------------------------------\n".format(
        correct_preds, total_preds, val_loss, val_acc)
    print(valreport)

    if val_acc > best_val_acc:
        print('Updating the model')
        best_val_acc = val_acc
        checkpoint = {
            "epoch": epoch,
            "model_state_dict": Net.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "train_acc": train_acc,
            "valid_acc": val_acc
        }
        torch.save(checkpoint, os.path.join(expDir, 'checkpoint.tar'))
        torch.save(Net, os.path.join(expDir, 'model.pth'))

    correct_preds = 0
    total_preds = 0

print('Finished Training')


[1,   300] loss: 4.987
[1,   600] loss: 2.408
[1,   900] loss: 1.585
[1,  1200] loss: 1.181
[1,  1500] loss: 0.939
[1,  1800] loss: 0.786
[1,  2100] loss: 0.674
[1,  2400] loss: 0.584
[1,  2700] loss: 0.524
[1,  3000] loss: 0.469
[1,  3300] loss: 0.426
[1,  3600] loss: 0.388
[1,  3900] loss: 0.359
[1,  4200] loss: 0.335
[1,  4500] loss: 0.312
[1,  4800] loss: 0.290
[1,  5100] loss: 0.273
[1,  5400] loss: 0.257
[1,  5700] loss: 0.247
[1,  6000] loss: 0.233
[1,  6300] loss: 0.222
[1,  6600] loss: 0.210
Training Time for Epoch :0 
 613.9519198000198 seconds

Training Accuracy : 
 correct predictions  : 1379 
 total predictions : 54873 
 Training Accuracy : 2.5130756474040057 
 ------------------------

Validation Time for Epoch :0 
 67.33142370000132 seconds

Validation Accuracy : 
 correct predictions  : 516 
 total predictions : 15678 
 Val-Loss : 4.643454030272091 
 Val-Accuracy :3.2912361270570227----------------------------------

Updating the model
[2,   300] loss: 4.658
[2,   600] 

[9,  6600] loss: 0.194
Training Time for Epoch :8 
 293.7249832999951 seconds

Training Accuracy : 
 correct predictions  : 2988 
 total predictions : 54873 
 Training Accuracy : 5.445300967689028 
 ------------------------

Validation Time for Epoch :8 
 35.59459750002134 seconds

Validation Accuracy : 
 correct predictions  : 839 
 total predictions : 15678 
 Val-Loss : 4.376234903887897 
 Val-Accuracy :5.351447888761322----------------------------------

Updating the model
[10,   300] loss: 4.346
[10,   600] loss: 2.141
[10,   900] loss: 1.434
[10,  1200] loss: 1.073
[10,  1500] loss: 0.860
[10,  1800] loss: 0.719
[10,  2100] loss: 0.619
[10,  2400] loss: 0.537
[10,  2700] loss: 0.483
[10,  3000] loss: 0.432
[10,  3300] loss: 0.392
[10,  3600] loss: 0.359
[10,  3900] loss: 0.330
[10,  4200] loss: 0.310
[10,  4500] loss: 0.290
[10,  4800] loss: 0.270
[10,  5100] loss: 0.251
[10,  5400] loss: 0.237
[10,  5700] loss: 0.230
[10,  6000] loss: 0.216
[10,  6300] loss: 0.206
[10,  6600] los

[18,   300] loss: 4.167
[18,   600] loss: 2.034
[18,   900] loss: 1.370
[18,  1200] loss: 1.022
[18,  1500] loss: 0.823
[18,  1800] loss: 0.691
[18,  2100] loss: 0.592
[18,  2400] loss: 0.519
[18,  2700] loss: 0.462
[18,  3000] loss: 0.415
[18,  3300] loss: 0.375
[18,  3600] loss: 0.342
[18,  3900] loss: 0.317
[18,  4200] loss: 0.295
[18,  4500] loss: 0.279
[18,  4800] loss: 0.260
[18,  5100] loss: 0.239
[18,  5400] loss: 0.227
[18,  5700] loss: 0.221
[18,  6000] loss: 0.209
[18,  6300] loss: 0.197
[18,  6600] loss: 0.185
Training Time for Epoch :17 
 296.056631799991 seconds

Training Accuracy : 
 correct predictions  : 4296 
 total predictions : 54873 
 Training Accuracy : 7.828986933464545 
 ------------------------

Validation Time for Epoch :17 
 35.32081979999202 seconds

Validation Accuracy : 
 correct predictions  : 1132 
 total predictions : 15678 
 Val-Loss : 4.239010590803777 
 Val-Accuracy :7.220308712846027----------------------------------

Updating the model
[19,   300] 

[26,  4800] loss: 0.256
[26,  5100] loss: 0.236
[26,  5400] loss: 0.223
[26,  5700] loss: 0.217
[26,  6000] loss: 0.206
[26,  6300] loss: 0.194
[26,  6600] loss: 0.182
Training Time for Epoch :25 
 294.7763859000115 seconds

Training Accuracy : 
 correct predictions  : 4582 
 total predictions : 54873 
 Training Accuracy : 8.35019043974268 
 ------------------------

Validation Time for Epoch :25 
 34.19455309998011 seconds

Validation Accuracy : 
 correct predictions  : 1162 
 total predictions : 15678 
 Val-Loss : 4.2288304394735245 
 Val-Accuracy :7.411659650465621----------------------------------

[27,   300] loss: 4.073
[27,   600] loss: 2.013
[27,   900] loss: 1.347
[27,  1200] loss: 1.004
[27,  1500] loss: 0.811
[27,  1800] loss: 0.678
[27,  2100] loss: 0.582
[27,  2400] loss: 0.513
[27,  2700] loss: 0.454
[27,  3000] loss: 0.409
[27,  3300] loss: 0.370
[27,  3600] loss: 0.336
[27,  3900] loss: 0.312
[27,  4200] loss: 0.290
[27,  4500] loss: 0.273
[27,  4800] loss: 0.256
[27,  

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(loss_history)
plt.savefig(lossDir + '/loss_history.png')

In [ ]:
# path='/content/drive/MyDrive/schubert/sine_model.pth'
# path='/content/drive/MyDrive/schubert/lstmmodel1.pth'
path='/content/drive/MyDrive/schubert/sine_wavenet1.pth'
torch.save(Net.state_dict(), path)

In [ ]:
# path='/content/drive/MyDrive/schubert/sine_model.pth'
# path='/content/drive/MyDrive/schubert/lstmmodel1.pth'
checkpoint_path='/home/god-particle/Desktop/Music_Gen_AI/trained_model_cache/29_08_2021_23_57/checkpoint.tar'
Net = LSTM.Sequence(len(unique_notes),32);
checkpoint = torch.load(checkpoint_path)
Net.load_state_dict(checkpoint["model_state_dict"])
print(checkpoint["epoch"])
# Net = Wavenet(len(unique_x),blocks=4,layers = 2)
# Net.load_state_dict(torch.load(path))

correct_preds = 0
total_preds = 0
with torch.no_grad():
  for i, data in enumerate(validationloader, 0):
    input , label = data
    output = Net(input,input.shape[0])
    total_preds += input.shape[0]
    correct_preds += torch.sum(torch.argmax(output,1) == label)
    

report ="correct predictions  : {} \n total predictions : {} \n Accuracy : {}".format(correct_preds,total_preds, correct_preds/total_preds *100)
print(report)


### **Testing the Model by training sinosoids**



*   Uncomment the code chunk below to test o sinosoids 
*   dont run above code if training for sinosoids. Variable names are mostly same



In [24]:
# time = np.linspace(1,10,10000)
# # print(time)
# y = np.sin(20*time)
# # plt.figure();
# # plt.plot(y)
# func = np.round(y,decimals=2)
# # print(y[:100])
# no_of_timesteps = 32
# x = []
# y = []


# for i in range(0, len(func) - no_of_timesteps-1, 1):
    
#     #preparing input and output sequences
#     input_ = func[i:i + no_of_timesteps]
#     output = func[i+no_of_timesteps+1]
    
#     x.append(input_)
#     y.append(output)
        
# x=np.array(x)
# y=np.array(y)

# unique_x = list(set(x.ravel()))
# x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))
# x_seq=[]
# for i in x:
#     temp=[]
#     for j in i:
#         #assigning unique integer to every note
#         temp.append(x_note_to_int[j])
#     x_seq.append(temp)
    
# x_seq = np.array(x_seq)

# unique_y = list(set(y))
# y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y)) 
# y_seq=np.array([x_note_to_int[i] for i in y])

In [25]:
%run MusicDataset.py
